# ライブラリインポート

In [ ]:
!pip install japanize_matplotlib | tail -n 1
!pip install torchviz | tail -n 1
!pip install torchinfo | tail -n 1
w = !apt install tree
print(w[-2])

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
from IPython.display import display
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchviz import make_dot
from tqdm.notebook import tqdm
from torchvision.transforms.transforms import RandomErasing
from torchvision import models

# 共通関数のダウンロード
!git clone https://github.com/makaishi2/pythonlibs.git

# 共通関数のロード
from pythonlibs.torch_lib1 import *

# 共通関数の存在チェック
print(README)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

Setting up tree (1.7.0-5) ...
Cloning into 'pythonlibs'...
remote: Enumerating objects: 108, done.
remote: Total 108 (delta 0), reused 0 (delta 0), pack-reused 108
Receiving objects: 100% (108/108), 21.11 MiB | 32.86 MiB/s, done.
Resolving deltas: 100% (49/49), done.
Common Library for PyTorch
Author: M. Akaishi
cpu


# ファイルアップロード

In [ ]:
from google.colab import files
# zipファイルを指定する
files.upload()

In [ ]:
# データ解凍
w = !unzip -o crunch.zip

# 結果確認
print(w[-1])

In [ ]:
# 解凍ファイルのtree表示
!tree crunch

# Transform定義

In [ ]:
# transforms定義

# 検証データ
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

# 訓練データ
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=5),
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False)
])

# Dataset定義

In [ ]:
# データセット定義

data_dir = "crunch"

import os
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "val")

classes = ["ng", "ok"]

train_data = datasets.ImageFolder(train_dir, transform = train_transform)
train_data2 = datasets.ImageFolder(train_dir, transform = test_transform)
test_data = datasets.ImageFolder(test_dir, transform = test_transform)

In [ ]:
# データ件数確認
print(f"学習データ: {len(train_data)}件")
print(f"検証データ: {len(test_data)}件")

# DataLoader定義

In [ ]:
# データローダー定義
batch_size = 5

train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_data, batch_size = batch_size, shuffle = False)

# 訓練表示用
train_loader2 = DataLoader(train_data2, batch_size = batch_size, shuffle = False)

# 検証表示用
test_loader2 = DataLoader(test_data, batch_size = 10, shuffle = True)

# 訓練・検証データ確認

In [ ]:
# 訓練データ表示
show_images_labels(train_loader2, classes, None, None)

In [ ]:
# 検証データ表示
torch_seed()
show_images_labels(test_loader2, classes, None, None)

# Model定義

## VGG19_BN

In [ ]:
# モデル定義

net = models.vgg19_bn(pretrained = True)

for param in net.parameters():
  param.requires_grad = False

  # 乱数初期化
  torch_seed()

  # 最終ノード出力を2に変更する
  in_features = net.classifier[6].in_features
  net.classifier[6] = nn.Linear(in_features, 2)

  # AdaptiveAvgPool2d関数の取り外し
  net.avgpool = nn.Identity()

  # GPU利用
  net = net.to(device)

  # 学習率
  lr = 0.001

  # 損失関数
  criterion = nn.CrossEntropyLoss()

  # 最適化関数
  # パラメータ修正の対象を最終ノードに限定
  optimizer = optim.SGD(net.classifier[6].parameters(), lr=lr, momentum=0.9)

  history = np.zeros((0, 5))

In [ ]:
# 学習実行

num_epochs = 10

history = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history)

In [ ]:
# 結果
evaluate_history(history)

In [ ]:
# 予測結果表示
torch_seed()
show_images_labels(test_loader2, classes, net, device)

## Densenet201

In [ ]:
# モデル定義
net = models.densenet201(pretrained = True)

In [ ]:
print(net)

In [ ]:
for param in net.parameters():
  param.requires_grad = False

  # 乱数初期化
  torch_seed()

  # 最終ノード出力を2に変更する
  in_features = net.classifier.in_features
  net.classifier = nn.Linear(in_features, 2)

  # AdaptiveAvgPool2d関数の取り外し
  net.avgpool = nn.Identity()

  # GPU利用
  net = net.to(device)

  # 学習率
  lr = 0.001

  # 損失関数
  criterion = nn.CrossEntropyLoss()

  # 最適化関数
  # パラメータ修正の対象を最終ノードに限定
  optimizer = optim.SGD(net.classifier.parameters(), lr=lr, momentum=0.9)

  history = np.zeros((0, 5))

In [ ]:
# 学習実行

num_epochs = 10

history = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history)

In [ ]:
evaluate_history(history)

In [ ]:
# 予測結果表示
torch_seed()
show_images_labels(test_loader2, classes, net, device)